# Introduction 

Automatic differentiation (AD) is a set of techniques to numerically evaluate the derivative of a function specified by a computer program. In mathematics and computer fields, a large number of numerical methods require derivatives to be calculated(Margossian, 2019). For example, in machine learning, gradient plays an essential role. Neural networks are able to gradually increase accuracy with every training session through the process of gradient descent(Wang, 2019).  In physics, differentiation is needed to quantify the rate of change in the equations of motion. However, for many problems, calculating derivatives analytically is both time consuming and computationally impractical. Automatic differentiation can gives exact answers in constant time(Saroufim, 2019), which is a powerful tool to automate the calculation of derivatives, especially when differentiating complex algorithms and mathematical functions(Margossian, 2019).  

In this library, we will describe the mathematical background and concepts in the Background section, explain the software organization, and discuss the implementation of the forward mode of automatic differentiation.



# Background

Automatic differentiation assumes that we are working with a differentiable function composed of a finite number of elementary functions and operations with known symbolic derivatives. The table below shows some examples of elementary functions and their respective derivatives:

| Elementary Function    | Derivative              |
| :--------------------: | :----------------------:|
| $x^3$                  | $3 x^{2}$               | 
| $e^x$                  | $e^x$                   |
| $\sin(x)$              | $\cos(x)$               |
| $\ln(x)$               | $\frac{1}{x}$           |

Given a list of elementary functions and their corresponding derivatives, the automatic differentiation process involves the evaluations of the derivatives of complex compositions of these elementary functions through repeated applications of the chain rule:

$ \frac{\partial}{\partial x}\left[f_1 \circ (f_2 \circ \ldots (f_{n-1} \circ f_n)) \right] = 
\frac{\partial f_1}{\partial f_2} \frac{\partial f_2}{\partial f_3} \ldots \frac{\partial f_{n-1}}{\partial f_n}\frac{\partial f_n}{\partial x}$

This process can be applied to the evaluation of partial derivatives as well thus allowing for computing derivatives of multivariate and vector-valued functions.

## Computational Graph

The forward mode automatic differentiation process described above can be visualized in a computational graph, a directed graph with each node corresponding to the result of an elementary operation.

For example, consider the simple problem of evaluating the following function and its derivative at $x=2$:
$$
f(x) = x^3 +2x^2
$$
The evaluation of this function can be represented by the computational graph below where the numbers in orange are the function values and the numbers in blue are the derivatives evaluated after applying each elementary operation:

<img src="https://raw.githubusercontent.com/anita76/playground/master/src/ex_comp_graph.png" width="75%" />

# How to Use West Coast AD

Having installed the package, users can evaluate derivatives of functions written in terms of WestCoastAD variables. For instance, the derivative of $f(x) = x^3+ 2x^2$ can be evaluated as follows:

In [ ]:
# 1) import WestCoastAD module
import WestCoastAD as ad
# 2) define the variables of your function as WestCoastAD variables 
#    with the value at which you want to evaluate the derivative and 
#    a derivative seed value
x = ad.Variable(value=2, derivative_seed=1) 
# 3) define your function in terms of the WestCoastAD variable objects
f = x**3 + 2* x**2
# 4) access the computed derivative and the value of the function 
f.value
f.derivative

# Software Organization
## Directory Structure
```bash
WestCoastAD/
             __init__.py  #Initialize the main package
             LICENSE
             README.MD
             docs/         #Subpackage for package docs
                     __init__.py
                     docs.ipynb
                     ...
             
             installation/  #Subpackage for install mode
                     __init__.py
                     setup.py
                     ...             
             src/  
                             forward_mode/  #Subpackage for forward mode
                                     __init__.py
                                     forward_mode_module.py
                                     ...

                             extension/  #Subpackage for future extention
                                     __init__.py
                                     extension_module.py
                                     ...
             
             tests/         #Subpackage for test
                             __init__.py
                             forward_mode_test/  #Subpackage for forward mode
                                     __init__.py
                                     forward_mode_test.py                
                                     ...

                             extension_test/  #test for future extension
                                     __init__.py
                                     extension_test.py
                                     ...
             
                     ...
```
## Modules and Basic Functionality

#### Docs Module
* This module will contain the documentation for the package. The Docs will include the information about the package including the purpose, the organization of the software, how to use the package and how to install the package.

#### Setup Module
* This module will contain configurations to allow users to install our package. 

#### Forward Mode Module
* This module will have all the code for implementing the forward mode of automatic differentiation. 
The module will contain a Variable Class that will take the value to be evaluated and the derivative seed value. 
It will return the derivative of a function that is defined in terms of the variable class.

#### Extension Mode Module
* The extension mode will contain our future expansion for the AD package.

#### Test Modules
* This module will contain all the test that will be performed before the build. It will contain unit test and acceptance test. 
We will use pytest in our package and check our coverage with CodeCov.


## Test Suite
* As we are using PyScaffold our test will have their own directory and we will utilize pytest to perform our test. 
Also, we use CodeCov to give us a percentage of code coverage

## Travis CI
* Travis CI allows the build to be automated so it does not require a constant approval of updates by team members. 
It requires that all pull request be tested and if all the test is passed it will update the build automatically.  
Travis CI also contains a badge that will inform the user if the package is working or if it is down.

## CodeCov
* CodeCov will inform the how much of the lines in our package are covered by test. 
This allows us to identify where test need to be added and will help discover possible bugs in the code. 
CodeCov also contains a badge that will inform the user a percentage of how much of the code is covered by test.

## Distribution
* For this package, we will use Github to distribute our package. This will require the user to clone our repo and follow our instructions to use the package. We will look into registering with conda-forge and dockerize.

## Packaging
* We will use the packaging framework PyScaffold as it is a streamlined way to setup our packaging. PyScaffold will setup our folder structure and make the configuration simplified. We will have to update the config once we want to package our library. As per the requirements, we will use the Spinx module to setup our documentation and pytest for the testing.




# Software Implementation

#### Core Data structures

As per our current design, we plan to use the following data structures -

1. Numpy arrays

These will be used to save the values and derivatives in our Variable class. If the array has length 1, we are dealing with scalar values and if it has a longer length we are dealing with vectors (e.g. when dealing with vector valued functions)

2. `Variable` objects

`Variable` class can be used to instantiate gradient objects.
These objects should be used to express the scalar or vector function to be differentiated. We overload the primitive mathematical operations to evaluate the derivatives in addition to function operation values. This class has the following attributes and methods:


| Class Attributes        | Usage         |
| ----------------------- |:-------------|
| `value`                 | Value of the variable at which the derivative has to be evaluated (an array).|
| `derivative_seed`      | Seed value of the derivative for differentation. Defaults to 1 (an array).             |
| `derivative`            | Returns the derivative value  (an array)        |

<br>

| Class Methods           | Usage|
| ----------------------- |:-------------|
| `mul`                   | Operator overloaded `*` with product rule in differentiation     |
| `pow`                   | Operator overloaded `**` with power rule in differentiation. For instance x`**`3 returns 2`*`x`*`2      |
| `sqrt`                  | Square root is a special case of pow and therefore can be implemented by calling __pow__ with exponent `1\2`    |
| `sin`                   | evaluates `sin` and `cos` (the derivative of sine function)    |
| `cos`                   | evaluates `cos` and its derivative -`sin`    |
| `exp`                   | evaluates `exp` and its derivative `exp`  |
| `log`                   | evaluates `log` and its derivative `1/arg`     |
| `jacobian_matrix`       | Returns the computed jacobian matrix             |
| `differentiate`         | Specify the mode (defaulting to forward mode ) and returns the derivative vector/scalar            |


Note: other mathematical operations (e.g. summation, division, etc) will have corresponding function definitions as well

#### External Dependencies
In addition to the built-in libraries, we will use `numpy` package for implementation.

# Feedback

## Milestone 1

Introduction
- How does AD compare and improve from other similar methods?

Background
- The flow could be enhanced by presenting an evaluation trace along with the computational graph.

How to use
- How do you install the package? By cloning your repo? Add more info on this here.
- Add the expected output from your example.

Software Organization
- Looks good!

Implementation
- So you have one class to override all the mathematical functions and to take the derivative? Or is the Variable class separate to the class methods? Elaborate further.


# References
-  Margossian, C. C. (2019). A review of automatic differentiation and its efficient implementation. Wiley Interdisciplinary Reviews: Data Mining and Knowledge Discovery, 9(4), 1–32. https://doi.org/10.1002/widm.1305

- Saroufim, M. (2019, November 13). Automatic Differentiation Step by Step. Retrieved October 15, 2020, from https://marksaroufim.medium.com/automatic-differentiation-step-by-step-24240f97a6e6

- Wang, C. (2019, March 3). Automatic Differentiation, Explained. Retrieved October15, 2020,from https://towardsdatascience.com/automatic-differentiation-explained-	b4ba8e60c2ad

- Automatic Differentiation Background. Retrived October 20, 2020, from https://www.mathworks.com/help/deeplearning/ug/deep-learning-with-automatic-differentiation-in-matlab.html